In [1]:
import keras
import os
import sys
from allennlp.commands.elmo import ElmoEmbedder
import numpy as np
import random
from keras.models import Sequential
import pandas as pd
from keras.layers import Dense, Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D, Activation, Dropout
from sklearn import preprocessing
from allennlp.data.tokenizers.word_splitter import SpacyWordSplitter
from allennlp.data.tokenizers import word_tokenizer
import multiprocessing as mp

Using TensorFlow backend.


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
!wget https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/contributed/pt/elmo_pt_weights.hdf5
!wget https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/contributed/pt/elmo_pt_options.json
!wget https://github.com/explosion/spacy-models/releases/download/pt_core_news_sm-2.1.0/pt_core_news_sm-2.1.0.tar.gz

!pip install pt_core_news_sm-2.1.0.tar.gz

--2019-04-15 02:23:53--  https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/contributed/pt/elmo_pt_weights.hdf5
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.192.144
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.192.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 374434776 (357M) [binary/octet-stream]
Saving to: ‘elmo_pt_weights.hdf5.1’

elmo_pt_weights.hdf 100%[===================>] 357.09M  1.20MB/s    in 4m 39s  

2019-04-15 02:28:33 (1.28 MB/s) - ‘elmo_pt_weights.hdf5.1’ saved [374434776/374434776]

--2019-04-15 02:28:33--  https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/contributed/pt/elmo_pt_options.json
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.160.52
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.160.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 546 [text/plain]
Saving to: ‘elmo_pt_

In [2]:
data = pd.read_csv("Tweets_Mg.csv", encoding='utf-8')[['Text','Classificacao']]
data.columns = ['text','label']

op = "elmo_pt_options.json"
wg = "elmo_pt_weights.hdf5"
elmo = ElmoEmbedder(options_file=op, weight_file=wg)
word_splitter = SpacyWordSplitter(language='pt_core_news_sm')

In [7]:
??SpacyWordSplitter

In [ ]:
def foo(elmo_embedding):
    avg_elmo_embedding = np.average(elmo_embedding, axis=0)
    return avg_elmo_embedding


def get_elmo_embeddings(data,elmo,op = "elmo_pt_options.json",wg = "elmo_pt_weights.hdf5",
                        text_column='text',label_column='label',max_tokens = 100):
    tk = word_tokenizer.WordTokenizer(word_splitter=word_splitter)
    
    tokens = tk.batch_tokenize(data[text_column])
    idx = []

    for i in range(len(tokens)):
        idx.append([x.idx for x in tokens[i]])
        tokens[i] = [x.text for x in tokens[i]]
    
    embeddings = []
    documentIdx = 0
    num_sentences = len(tokens)
    #embeddings = list(pool.map(foo,elmo.embed_sentences(tokens))) # or use map
    for elmo_embedding in elmo.embed_sentences(tokens,batch_size=50):  
        # Average the 3 layers returned from ELMo
        avg_elmo_embedding = np.average(elmo_embedding, axis=0)

        embeddings.append(avg_elmo_embedding)

        # Some progress info
        documentIdx += 1
        percent = 100.0 * documentIdx / num_sentences
        line = '[{0}{1}]'.format('=' * int(percent / 2), ' ' * (50 - int(percent / 2)))
        status = '\r{0:3.0f}%{1} {2:3d}/{3:3d} sentences'
        sys.stdout.write(status.format(percent, line, documentIdx, num_sentences))
    
    paded_embeddings = keras.preprocessing.sequence.pad_sequences(embeddings,maxlen=max_tokens,dtype='float32')
    return paded_embeddings, data[label_column]

In [49]:
train_x, train_y= get_elmo_embeddings(data[:1000],elmo)

print("Shape Train X:", train_x.shape)
print("Shape Train Y:", train_y.shape)

100%[==================================================] 1000/1000 sentencesPading Matrix
Shape Train X: (1000, 100, 1024)
Shape Train Y: (1000,)


In [50]:
# Simple model for sentence / document classification using CNN + global max pooling
model = Sequential()
model.add(Conv1D(filters=250, kernel_size=3, padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dense(250, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [51]:
le = preprocessing.LabelEncoder()
le.fit(train_y)

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)
y_enc = np.asarray(encode(le, train_y))


In [52]:
model.fit(train_x, y_enc,validation_split=0.5, epochs=10, batch_size=10)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 500 samples, validate on 500 samples
Epoch 1/10
500/500 [==============================] - 5s 10ms/step - loss: 0.6121 - acc: 0.8100 - val_loss: 0.2912 - val_acc: 0.8500
Epoch 2/10
500/500 [==============================] - 4s 8ms/step - loss: 0.2416 - acc: 0.9100 - val_loss: 0.1318 - val_acc: 0.9680
Epoch 3/10
500/500 [==============================] - 4s 8ms/step - loss: 0.1002 - acc: 0.9580 - val_loss: 0.6555 - val_acc: 0.6880
Epoch 4/10
500/500 [==============================] - 4s 8ms/step - loss: 0.1211 - acc: 0.9680 - val_loss: 0.2220 - val_acc: 0.9500
Epoch 5/10
500/500 [==============================] - 4s 8ms/step - loss: 0.0740 - acc: 0.9800 - val_loss: 0.4475 - val_acc: 0.8060
Epoch 6/10
500/500 [==============================] - 4s 8ms/step - loss: 0.0555 - acc: 0.9840 - val_loss: 0.6895 - val_acc: 0.8000
Epoch 7/10
500/500 [===============

In [53]:
predicts = model.predict(train_x, batch_size=32)
y_test = decode(le, y_enc)
y_preds = decode(le, predicts)

from sklearn import metrics

print(metrics.confusion_matrix(train_y, y_preds))

print(metrics.classification_report(train_y, y_preds))

[[390   6   0]
 [ 72 509   1]
 [  0   6  16]]
              precision    recall  f1-score   support

    Negativo       0.84      0.98      0.91       396
      Neutro       0.98      0.87      0.92       582
    Positivo       0.94      0.73      0.82        22

   micro avg       0.92      0.92      0.92      1000
   macro avg       0.92      0.86      0.88      1000
weighted avg       0.92      0.92      0.92      1000

